In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim
import numpy as np
import traci
import optparse
import os
import sys
import pandas as pd
import json
import tqdm
from collections import deque
import random

from sumolib import checkBinary  # noqa
import traci  # noqa

#Only works for micro simulation (1 traffic light)
class SumoEnvrionment:
    def __init__(self, gui = True, total_phases, buffer_size = 12):
        self.total_phases = total_phases
        self.buffer_size = buffer_size
        self.gui = gui

        self.lanes_dict = {}
        self.lane_IDs = traci.lanearea.getIDList()
        for lane in self.lane_IDs:
            self.lanes_dict[lane]=[]

        self.start_program()

    def start_program(self):
        options = self.get_options()
        options.nogui = self.gui

        # this script has been called from the command line. It will start sumo as a server, then connect and run
        if options.nogui:
            sumoBinary = checkBinary('sumo')
        else:
            sumoBinary = checkBinary('sumo-gui')

        # we need to import python modules from the $SUMO_HOME/tools directory
        if 'SUMO_HOME' in os.environ:
            tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
            sys.path.append(tools)
        else:
            sys.exit("please declare environment variable 'SUMO_HOME'")

        traci.start([sumoBinary, "-c", "Simulation_Environment\Main Route Simulation\osm.sumocfg",
                             "--tripinfo-output", "Data\\tripinfo.xml"])

    def json_loader(self, path):
        with open(path,'r') as f:
            all_data ="".join([i for i in f])
            file = json.loads(all_data)
        return file
    
    def get_options(self):
        optParser = optparse.OptionParser()
        optParser.add_option("--nogui", action="store_true",
                            default=False, help="run the commandline version of sumo")
        options, args = optParser.parse_args()
        return options

    def update_lane_data(self):
        for lane in self.lane_IDs:
            lane_data = traci.lanearea.getLastStepHaltingNumber(lane)
            self.lanes_dict[lane].append(lane_data)
        return self.lanes_dict
    
    def save_vehicle_ids(self, lanes_dict):
        vehicle_ids=pd.DataFrame(lanes_dict)
        vehicle_ids['Steps'] = np.arange(0,len(lanes_dict[lane_IDs[0]]),1)
        vehicle_ids.to_csv('test.csv')

    def get_reward(self):
        all_speed = []
        e2_detectors = traci.lanearea.getIDList()

        for detector in e2_detectors:
            speed = traci.lanearea.getLastStepMeanSpeed(detector)
            all_speed.append(speed)
        
        return np.array(speed).mean()

    def get_state(self):
        e2_detectors = traci.lanearea.getIDList()

        queues = [traci.lanearea.getLastStepVehicleNumber(detector) for detector in e2_detectors] #gets the queus in detectors
        traffic_light = traci.trafficlight.getIDList()[0] # since there is only one traffic light

        tl_phase = traci.trafficlight.getPhase(traffic_light)
        one_hot_vector_tl_phase = np.eye(self.total_phases)[tl_phase]

        return np.hstack([queues, one_hot_vector_tl_phase])
    
    def action(self, action):
        self.lanes_dict = self.update_lane_data()
        vehicle_ids=pd.DataFrame(self.lanes_dict)
        vehicle_ids['Steps'] = np.arange(0,len(self.lanes_dict[self.lane_IDs[0]]),1)
        vehicle_ids.to_csv('test.csv')
        traci.trafficlight.setPhase(action)
        
        for i in range(12):
            traci.simulation.step()
    
    def is_done(self):
        return traci.simulation.getMinExpectedNumber() == 0


class Net(nn.Mdule):
    def __init__(self, total_phases):
        super().__init__()
        self.total_phases = total_phases
        self.fc1 = nn.Linear(8,1000)
        self.fc2 = nn.Linear(1000,500)
        self.fc3 = nn.Linear(500,self.total_phases)

    def forward(self, x):
        #Neural Network Forward Pass Layers
        x = nn.ReLU(self.fc1(x))
        x = nn.BatchNorm1d(x)
        x = nn.ReLU(self.fc2(x))
        x = nn.Relu(self.fc3(x))
        return x

def train(total_phases = 4, train=True, epochs = 80, mem_size = 1000, batch_size = 200, sync_freq = 500, epsilon = 0.2, discount_factor=0.9, learning_rate = 1e-3):
    gpu = torch.device('cuda:0')
    cpu = torch.device('cpu:0')
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    net = Net()
    target_net = Net()
    replay = deque(maxlen=mem_size)

    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)
    losses=[]

    for i in tqdm(range(epochs)):
        env = SumoEnvrionment(total_phases)
        state1 = env.get_state()

        step = 0
        done = False

        while not done:
            step += 1
            qval = net(state1)
            qval_ = qval.data.numpy()
            if random.random() < epsilon:
                action = np.random.randint(0,4)
            else:
                action = np.argmax(qval_)

            env.action(action)
            state2 = env.get_state()
            reward = env.get_reward()
            done = env.is_done()
            exp = [state1, action, reward, state2, done]
            replay.append(exp)

            if len(replay) > batch_size:
                minibatch = random.sample(replay, batch_size).to(gpu)
                state1_batch = torch.cat([s1 for (s1,a,r,s2,d) in minibatch]).to(gpu)
                action_batch = torch.Tensor([a for (s1,a,r,s2,d) in minibatch]).to(gpu)
                reward_batch = torch.Tensor([r for (s1,a,r,s2,d) in minibatch]).to(gpu)
                state2_batch = torch.cat([s2 for (s1,a,r,s2,d) in minibatch]).to(gpu)
                done_batch = torch.Tensor([d for (s1,a,r,s2,d) in minibatch]).to(gpu)

                net.to_gpu(gpu)
                Q1 = net(state1_batch)

                target_net.to_gpu(gpu)
                with torch.nograd():
                    Q2 = target_net(state1_batch)
                
                Y = reward_batch + ((1-done_batch)*discount_factor*torch.max(Q2, dim=1)[0])
                X = Q1.gather(dim=1,index=action_batch.long().unsqueeze(dim=1)).squeeze()

                loss = loss_fn(X,Y.detach())
                print(i, loss.item())
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                losses.append(loss.item())
                state1 = state2

                if step == sync_freq:
                    target_net.load_state_dict(net.state_dict())
        traci.load()

    target_net.to(torch.device('cpu'))
    torch.save(target_net, Path('DQN_Model'))
    return target_net, losses

def graph_losses(losses, rolling_weight = 50):
    x = np.arange(0,len(losses),1)
    y = losses

    df = pd.DataFrame({'Epochs':x, 'Losses':y})
    df['Epochs'] = df['Epochs'].apply(lambda x: f"Epoch {x}")
    df['Rolling Losses'] = df['Losses'].rolling(rolling_weight).mean()

    fig, ax = plt.subplots()

    ax.plot(df['Epoch'], df['Losses'], label='Losses')
    ax.plot(df['Epoch'], df['Rolling Losses'], label='Rolling {rolling_weight} Epoch Average')
    
    condition = [True if i % rolling_weight == 0 else False for i in df['Epochs'].to_numpy()]
    ax.set_xticks(df['Epochs'][condition])

    ax.legend()
    plt.show()

if __name__ == '__main__':
    target_net, losses = train()
    graph_losses
    traci.close()
    


In [ ]:
class Net(nn.module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8,64)
        self.fc2 = nn.Linear(64,128)
        self.fc3 = nn.Linear(128,1)
    
    def forward(self, x):
        x = nn.ReLU(self.fc1(x))
        x = nn.ReLU(self.fc2(x))
        x = nn.ReLU(self.fc3(x))
        return x

x = torch.randn([1,8])

net = Net()



In [22]:
x = torch.Tensor([1,2,3,4,5])
x

tensor([1., 2., 3., 4., 5.])

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

x = np.arange(0,len(x),1)
y = np.random.randint(0,40,size=40)

df = pd.DataFrame({'Epochs':x, 'Losses':y})
df['Epochs'] = df['Epochs'].apply(lambda x: f"Epoch {x}")
df.head()

,Epochs,Losses
0,Epoch 0,31
1,Epoch 1,36
2,Epoch 2,22
3,Epoch 3,32
4,Epoch 4,3


In [5]:
df['Moving_Average'] = df['Losses'].rolling(7).mean()

In [15]:
condition = [True]*len(df['Epochs'])
df['Epochs'][condition]

0      Epoch 0
1      Epoch 1
2      Epoch 2
3      Epoch 3
4      Epoch 4
5      Epoch 5
6      Epoch 6
7      Epoch 7
8      Epoch 8
9      Epoch 9
10    Epoch 10
11    Epoch 11
12    Epoch 12
13    Epoch 13
14    Epoch 14
15    Epoch 15
16    Epoch 16
17    Epoch 17
18    Epoch 18
19    Epoch 19
20    Epoch 20
21    Epoch 21
22    Epoch 22
23    Epoch 23
24    Epoch 24
25    Epoch 25
26    Epoch 26
27    Epoch 27
28    Epoch 28
29    Epoch 29
30    Epoch 30
31    Epoch 31
32    Epoch 32
33    Epoch 33
34    Epoch 34
35    Epoch 35
36    Epoch 36
37    Epoch 37
38    Epoch 38
39    Epoch 39
Name: Epochs, dtype: object

In [7]:
import torch
torch.Tensor([8,5,4,2]).unsqueeze(0)[0]

tensor([8., 5., 4., 2.])

In [4]:
import optparse

# parser = optparse.OptionParser()
# parser.add_option("-S")
# option = parser.parse_args()
# print(option)
x=4
x

4